# IBIT Backtest: AMMA vs Optimized No-Leverage Combo (5 models)

Goal for this notebook:
- Build IBIT signals from **OU, Trend, Mining, ZScore, AMMA**
- Combine them with **long-only, sum-to-1 weights** (no leverage)
- Optimize combination to beat AMMA on Sharpe (in-sample)
- Plot equity and drawdown with hard metrics


In [ ]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = Path.cwd().resolve()
crypto_root = None
for c in [cwd, cwd / 'Crypto', cwd.parent, cwd.parent / 'Crypto', cwd.parent.parent / 'Crypto']:
    if (c / 'Data' / 'IBIT_Data.csv').exists():
        crypto_root = c
        break
if crypto_root is None:
    raise FileNotFoundError('Could not locate Crypto root with Data/IBIT_Data.csv')

if str(crypto_root) not in sys.path:
    sys.path.insert(0, str(crypto_root))

import config as cfg
from Backtest.amma import amma_from_ibit_csv
from Data.raw_data_loader import load_ibit_with_mining_cost
from Models.trend import trend_signal
from Models.mining import mining_signal
from Models.ou import ou_signal
from Models.zscore import zscore_signal


In [ ]:
# Helpers

def run_backtest_from_position(price: pd.Series, pos: pd.Series, fee_bps: float, slippage_bps: float) -> dict:
    price = price.astype(float)
    ret = price.pct_change().fillna(0.0)
    pos = pos.reindex(price.index).fillna(0.0).clip(0.0, 1.0)  # NO LEVERAGE, LONG-ONLY
    turnover = pos.diff().abs().fillna(pos.abs())
    costs = turnover * ((fee_bps + slippage_bps) / 1e4)
    net_ret = (pos * ret) - costs
    net_eq = (1.0 + net_ret).cumprod()
    return {"net_ret": net_ret, "net_equity": net_eq, "turnover": turnover}


def sr(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    if len(x) < 10:
        return np.nan
    v = x.std(ddof=1)
    if v == 0 or not np.isfinite(v):
        return np.nan
    return float(np.sqrt(periods) * x.mean() / v)


def sortino(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    dn = x[x < 0]
    if len(dn) < 2:
        return np.nan
    dv = dn.std(ddof=1) * np.sqrt(periods)
    if dv <= 0 or not np.isfinite(dv):
        return np.nan
    return float((x.mean() * periods) / dv)


def cagr(x: pd.Series, periods: int = 252) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    if len(eq) < 2:
        return np.nan
    yrs = len(eq) / periods
    return float(eq.iloc[-1] ** (1.0 / yrs) - 1.0) if yrs > 0 else np.nan


def maxdd(x: pd.Series) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    dd = eq / eq.cummax() - 1.0
    return float(dd.min())


def ann_vol(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    return float(np.sqrt(periods) * x.std(ddof=1)) if len(x) > 1 else np.nan


def win_rate(x: pd.Series) -> float:
    x = x.dropna()
    return float((x > 0).mean()) if len(x) else np.nan


def metric_row(name: str, r: pd.Series) -> dict:
    return {
        'Strategy': name,
        'CAGR': cagr(r),
        'Sharpe': sr(r),
        'Sortino': sortino(r),
        'MaxDD': maxdd(r),
        'Volatility': ann_vol(r),
        'WinRate': win_rate(r),
    }


In [ ]:
# Load IBIT + mining cost panel (IBIT trading dates)

panel = load_ibit_with_mining_cost(
    str(crypto_root / 'Data' / 'IBIT_Data.csv'),
    str(crypto_root / 'Data' / 'cleaned_crypto_data.csv'),
    forward_fill_mining_cost=True,
).sort_index().copy()

panel['close'] = panel['close'].astype(float)
model_df = pd.DataFrame(index=panel.index)
model_df['BTC-USD_close'] = panel['close']
model_df['COST_TO_MINE'] = panel['mining_cost']

ret = panel['close'].pct_change().fillna(0.0)


In [ ]:
# Build 5 model positions on IBIT panel (all clipped to [0,1])

# 1) AMMA
amma_df = amma_from_ibit_csv(
    str(crypto_root / 'Data' / 'IBIT_Data.csv'),
    momentum_weights={5: 0.25, 10: 0.25, 20: 0.25, 60: 0.25},
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
).rename(columns={'Date': 'date'}).set_index('date')
amma_pos = amma_df.reindex(panel.index)['amma_position'].fillna(0.0).clip(0.0, 1.0)

# 2) Trend
trend_pos = trend_signal(
    model_df,
    price_column='BTC-USD_close',
    fast_window=20,
    slow_window=100,
    long_only=True,
    leverage_aggressive=1.0,
    leverage_neutral=0.5,
    leverage_defensive=0.0,
).reindex(panel.index).fillna(0.0).clip(0.0, 1.0)

# 3) Mining
mining_pos = mining_signal(
    model_df,
    z_window=180,
    entry_z=0.8,
    exit_z=0.1,
    use_log_edge=True,
).reindex(panel.index).fillna(0.0).clip(0.0, 1.0)

# 4) OU
ou_pos = ou_signal(
    model_df['BTC-USD_close'],
    window=120,
    entry_z=1.5,
    exit_z=0.4,
    long_short=False,
).reindex(panel.index).fillna(0.0).clip(0.0, 1.0)

# 5) ZScore
zscore_pos = zscore_signal(
    model_df['BTC-USD_close'],
    resid_window=120,
    entry_z=1.25,
    exit_z=0.4,
    long_short=False,
    use_vol_target=False,
    max_leverage=1.0,
).reindex(panel.index).fillna(0.0).clip(0.0, 1.0)

positions = pd.DataFrame({
    'AMMA': amma_pos,
    'Trend': trend_pos,
    'Mining': mining_pos,
    'OU': ou_pos,
    'ZScore': zscore_pos,
}, index=panel.index)

positions.describe().T[['mean','std','min','max']]


In [ ]:
# Optimize no-leverage combo weights over all 5 models to beat AMMA on Sharpe

model_names = ['AMMA', 'Trend', 'Mining', 'OU', 'ZScore']

# model return streams after costs, each as standalone sleeve
model_returns = {}
for m in model_names:
    bt = run_backtest_from_position(panel['close'], positions[m], cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
    model_returns[m] = bt['net_ret']
model_returns = pd.DataFrame(model_returns, index=panel.index).dropna()

amma_sharpe = sr(model_returns['AMMA'])

# random simplex search (robust, no scipy dependency)
rng = np.random.default_rng(42)
best = {'w': np.array([1,0,0,0,0], dtype=float), 'sharpe': -np.inf, 'ret': model_returns['AMMA']}

# include corners and equal weight
candidates = [
    np.array([1,0,0,0,0], float),
    np.array([0,1,0,0,0], float),
    np.array([0,0,1,0,0], float),
    np.array([0,0,0,1,0], float),
    np.array([0,0,0,0,1], float),
    np.ones(5)/5,
]
# plus many random portfolios
for _ in range(120000):
    candidates.append(rng.dirichlet(np.ones(5)))

for w in candidates:
    combo_r = (model_returns[model_names] * w).sum(axis=1)
    s = sr(combo_r)
    if np.isfinite(s) and s > best['sharpe']:
        best = {'w': w, 'sharpe': float(s), 'ret': combo_r}

w_best = pd.Series(best['w'], index=model_names, name='weight')
combo_sharpe = best['sharpe']

print('AMMA Sharpe:', amma_sharpe)
print('Best combo Sharpe:', combo_sharpe)
print('Best weights (no leverage, sum=1):')
print(w_best)

if combo_sharpe <= amma_sharpe:
    print('WARNING: Best no-leverage 5-model combo did not exceed AMMA Sharpe in this run. Consider re-tuning model params.')
else:
    print('SUCCESS: Combo Sharpe is better than AMMA Sharpe.')


In [ ]:
# Final strategy comparison (orange line = AMMA, blue line = optimized combo)

combo_pos = (positions[model_names] * w_best.values).sum(axis=1).clip(0.0, 1.0)
combo_bt = run_backtest_from_position(panel['close'], combo_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
amma_bt = run_backtest_from_position(panel['close'], positions['AMMA'], cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
bh_bt = run_backtest_from_position(panel['close'], pd.Series(1.0, index=panel.index), fee_bps=0.0, slippage_bps=0.0)

rets = pd.DataFrame({
    'BuyHold_IBIT': bh_bt['net_ret'],
    'AMMA_IBIT': amma_bt['net_ret'],
    'Optimized5ModelCombo_IBIT': combo_bt['net_ret'],
}, index=panel.index)

summary = pd.DataFrame([
    metric_row('BuyHold_IBIT', rets['BuyHold_IBIT']),
    metric_row('AMMA_IBIT', rets['AMMA_IBIT']),
    metric_row('Optimized5ModelCombo_IBIT', rets['Optimized5ModelCombo_IBIT']),
]).set_index('Strategy').sort_values('Sharpe', ascending=False)
summary


In [ ]:
# Plots: equity and drawdown

eq = (1.0 + rets).cumprod()
dd = eq / eq.cummax() - 1.0

plt.figure(figsize=(12,5))
plt.plot(eq.index, eq['Optimized5ModelCombo_IBIT'], label='Optimized 5-Model Combo', linewidth=2.5, color='tab:blue')
plt.plot(eq.index, eq['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.2, color='tab:orange')
plt.plot(eq.index, eq['BuyHold_IBIT'], label='Buy & Hold', linewidth=1.8, linestyle='--', color='tab:gray')
plt.title('IBIT Equity: Optimized No-Leverage Combo vs AMMA vs Buy&Hold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(12,4))
plt.plot(dd.index, dd['Optimized5ModelCombo_IBIT'], label='Optimized 5-Model Combo', linewidth=2.0, color='tab:blue')
plt.plot(dd.index, dd['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.0, color='tab:orange')
plt.plot(dd.index, dd['BuyHold_IBIT'], label='Buy & Hold', linewidth=1.5, linestyle='--', color='tab:gray')
plt.title('IBIT Drawdown: Optimized No-Leverage Combo vs AMMA')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


In [ ]:
# Last cell requested: print exact selected model (commented)

exact_model_comment = (
    "# ----------------------------\n"
    "# FINAL SELECTED IBIT COMBO MODEL (NO LEVERAGE)\n"
    "# ----------------------------\n"
    "# Universe of models: AMMA, Trend, Mining, OU, ZScore\n"
    "# Position construction per model: clipped to [0, 1]\n"
    "# Combination rule:\n"
    "#   combo_pos_t = sum_i w_i * pos_i_t\n"
    "#   constraints: w_i >= 0, sum_i w_i = 1\n"
    "#   => no leverage at position or allocation layer\n"
    "# Optimization:\n"
    "#   maximize in-sample Sharpe of combo daily returns\n"
    "#   search: 120k random simplex portfolios + corner/equal portfolios\n"
    f"# Selected weights: AMMA={w_best['AMMA']:.6f}, Trend={w_best['Trend']:.6f}, Mining={w_best['Mining']:.6f}, OU={w_best['OU']:.6f}, ZScore={w_best['ZScore']:.6f}\n"
    f"# Sharpe(AMMA)={amma_sharpe:.6f}, Sharpe(Combo)={combo_sharpe:.6f}\n"
    "# ----------------------------"
)
print(exact_model_comment)
